In [1]:
import sys
from pathlib import Path
import pandas
import torch
from functools import partial
from IPython.display import Audio
import torchaudio

ROOT_PATH=Path("..").resolve().absolute()
DEVICE=torch.device("cpu")
sys.path.append(str(ROOT_PATH))

%load_ext autoreload
%autoreload 2
from evaluation.evaluate_separation import evaluate_tracks_chunks_simplified, sisnr
from main.dataset import is_silent
from evaluation.evaluate_separation import load_chunks
from main.separation import generate_mask_and_sources, ContextualSeparator, differential_with_dirac
from audio_diffusion_pytorch import KarrasSchedule
from script.misc import load_model, load_audio, load_context

In [13]:
#path = Path("/home/irene/Documents/audio-diffusion-pytorch-trainer/separations/complete_context_slakh_4stems_resamples")
path = Path("/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/context_slakh_glorious_star_num_steps=150_source_id=0_num_resamples=1_num_separations=3_num_gibbs_steps=1_hint_fixed_sources_idx=[]_use_heun=False_gibbs_sources=None_s_churn=10.0/sep_round_2")
results = evaluate_tracks_chunks_simplified(path, orig_sr=22050, eps=1e-8, compute_likelihood=False)

/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/context_slakh_glorious_star_num_steps=150_source_id=0_num_resamples=1_num_separations=3_num_gibbs_steps=1_hint_fixed_sources_idx=[]_use_heun=False_gibbs_sources=None_s_churn=10.0/sep_round_2


100%|███████████████████████████████████████| 2919/2919 [00:58<00:00, 49.81it/s]


In [14]:
results[["1", "2", "3", "4"]].mean()

1    13.754114
2    15.655675
3    10.692675
4    11.380738
dtype: float64

In [15]:
results[[ "1", "2", "3", "4"]].mean().mean()

12.870800662883838

In [10]:
results[["1", "2", "3", "4"]].mean().mean()

39.237333968200765

In [14]:
results[["1", "2", "3", "4"]].mean().mean()

16.451138272448283

In [4]:
path_part = Path("/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/context_slakh_num_steps=150_source_id=0_num_resamples=1_num_separations=1_num_gibbs_steps=1_hint_fixed_sources_idx=[]_use_heun=False/sep_round_0")
results_part = evaluate_tracks_chunks_simplified(path_part, orig_sr=22050, eps=1e-8, compute_likelihood=False)

100%|█████████████████████████████████████████| 200/200 [00:03<00:00, 61.18it/s]


In [40]:
root_dir = Path("../separations/debug_num_steps=150_source_id=0_num_resamples=1_num_separations=1_num_gibbs_steps=2_hint_fixed_sources_idx=[]_use_heun=False_gibbs_sources=None/sep_round_0")
csv_list = []

# Get all folders and their subfolders
folders = [f for f in root_dir.iterdir() if f.is_dir()]
subfolder_lists = [list(folder.glob("gibbs_*")) for folder in folders]

# Transpose the list of subfolders, so that each list contains subfolders with the same index across all folders
subfolders = list(map(list, zip(*subfolder_lists)))

for subfolder_group in subfolders:
    csv_frames = []
    for subfolder in subfolder_group:
        csv_path = subfolder / "metrics.csv"
        if csv_path.exists():
            df = pd.read_csv(csv_path)
            csv_frames.append(df)

    merged_csv = pd.concat(csv_frames, axis=0)
    csv_list.append(merged_csv)


In [41]:
csv_list[0][["bass", "drums", "guitar", "piano"]].mean().mean()

12.565043174661696

In [48]:
csv_list[0][0:20]

,Unnamed: 0,bass,drums,guitar,piano,start_sample,end_sample,gibbs_step,gibbs_fix_sources
0,0,23.292259,18.119215,9.639180,9.642313,0,88200,0,[]
1,1,22.888088,17.892803,9.419528,9.509614,44100,132300,0,[]
2,2,23.120014,18.502010,9.414358,9.608728,88200,176400,0,[]
3,3,23.393929,18.142958,10.624895,10.593834,132300,220500,0,[]
4,4,22.950926,17.531029,9.779424,9.769053,176400,264600,0,[]
0,0,16.884188,12.968410,8.772362,12.194743,0,88200,0,[]
1,1,17.513248,12.564702,6.225521,11.622501,44100,132300,0,[]
2,2,17.167387,12.967181,6.313715,11.616881,88200,176400,0,[]
3,3,16.018702,14.787477,9.259124,12.179578,132300,220500,0,[]
4,4,15.427570,13.032507,10.912449,16.388462,176400,264600,0,[]


In [26]:
ckpts_path = Path("/home/irene/Documents/audio-diffusion-pytorch-trainer/logs/ckpts")
model_cpu = load_context(ckpts_path / "avid-darkness-164_epoch=419-valid_loss=0.015.ckpt", "cpu", 4)
model = model_cpu.to("cuda:0")
mix, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/mix_gt.wav')
bass_gt, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/bass_gt.wav')
drums_gt, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/drums_gt.wav')
guitar_gt, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/guitar_gt.wav')
piano_gt, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/piano_gt.wav')

bass, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/bass_0_1.0.wav')
drums, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/drums_0_1.0.wav')
guitar, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/guitar_0_1.0.wav')
piano, sr = torchaudio.load('/home/giorgio_mariani/Documents/audio-diffusion-pytorch-trainer/separations/to_sep/piano_0_1.0.wav')

mix = mix.to("cuda:0")
bass_gt = bass_gt.to("cuda:0")
drums_gt = drums_gt.to("cuda:0")
guitar_gt = guitar_gt.to("cuda:0")
piano_gt = piano_gt.to("cuda:0")

bass = bass.to("cuda:0")
drums = drums.to("cuda:0")
guitar = guitar.to("cuda:0")
piano = piano.to("cuda:0")

separator = ContextualSeparator(
        model=model,
        stems=["bass", "drums", "guitar", "piano"],
        sigma_schedule=KarrasSchedule(sigma_min=1e-4, sigma_max=1., rho=7.0),
        differential_fn=differential_with_dirac,
        s_churn=20.,
        num_resamples=1,
        source_id=0,
    )
sources_idx=torch.arange(4, device=mix.device)
inpaint, inpaint_mask = generate_mask_and_sources(
    sources=torch.stack([s1, s1, s1, s1], dim=1),
    fixed_sources_idx = []
)
seps_dict = separator.separate_with_hint(
    mixture=mix.unsqueeze(0), 
    num_steps=150, 
    source_with_hint=inpaint,
    mask=inpaint_mask
)
seps = torch.cat([seps_dict["bass"], seps_dict["drums"], seps_dict["guitar"], seps_dict["piano"]], dim=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:04<00:00, 32.98it/s]


In [16]:
display(Audio(seps_dict["bass"].cpu().detach().reshape(1,-1), rate = sr1))

In [13]:
def sisnr(preds: torch.Tensor, target: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    alpha = (torch.sum(preds * target, dim=-1, keepdim=True) + eps) / (torch.sum(target**2, dim=-1, keepdim=True) + eps)
    target_scaled = alpha * target
    noise = target_scaled - preds
    s_target = torch.sum(target_scaled**2, dim=-1) + eps
    s_error = torch.sum(noise**2, dim=-1) + eps
    return 10 * torch.log10(s_target / s_error)

In [30]:
display(sisnr(seps[:, [0], :], bass_gt))
display(sisnr(seps[:, [1], :], drums_gt))
display(sisnr(seps[:, [2], :], guitar_gt))
display(sisnr(seps[:, [3], :], piano_gt))
display("---------------------------------")
display(sisnr(bass, bass_gt))
display(sisnr(drums, drums_gt))
display(sisnr(guitar, guitar_gt))
display(sisnr(piano, piano_gt))

tensor([[11.4627]], device='cuda:0')

tensor([[12.8218]], device='cuda:0')

tensor([[2.6365]], device='cuda:0')

tensor([[3.0730]], device='cuda:0')

'---------------------------------'

tensor([13.0442], device='cuda:0')

tensor([15.4238], device='cuda:0')

tensor([2.0097], device='cuda:0')

tensor([1.8324], device='cuda:0')

In [25]:
display(Audio(drums.reshape(1,-1).cpu().detach(), rate = sr))

In [425]:
def check_less_than(row, x):
    return (row < x).any()
filtered_df = results[results[["1", "2", "3", "4"]].apply(partial(check_less_than, x=-1), axis=1)]

In [26]:
filtered_df

,1,2,3,4,chunk_n,start_sample,end_sample
63,19.173681,20.200188,29.120819,-23.537796,70,176400,264600
84,12.387526,26.054993,3.503712,-1.317715,73,132300,220500
119,11.416879,11.989491,4.439496,-1.300331,53,132300,220500
120,8.911234,1.863950,6.519534,-2.738342,53,176400,264600
121,-38.845421,16.103876,9.332428,3.988993,55,0,88200
122,-35.838734,15.839083,9.543854,2.841281,55,44100,132300
123,-50.297733,16.280048,9.194588,1.779957,55,88200,176400
124,-66.790756,16.909756,9.413177,2.463057,55,132300,220500
125,-40.659088,14.461483,9.780258,4.342023,55,176400,264600
136,-16.584084,17.395786,0.354794,3.722847,90,0,88200


In [25]:
n = 185
chunk_folder = path / filtered_df["chunk_n"][n]
start = filtered_df["start_sample"][n]
end = filtered_df["end_sample"][n]
original_tracks, separated_tracks, sr = load_chunks(chunk_folder)
mixture = sum([original_tracks[str(k)] for k in range(1, 5)])

In [27]:
display(Audio(separated_tracks["4"][:, start:end].reshape(1,-1), rate = sr))
display(Audio(original_tracks["4"][:, start:end].reshape(1,-1), rate = sr))


In [241]:
torchaudio.save(Path("/home/giorgio_mariani/Desktop") / "ori4.wav", original_tracks["4"][:, start:end].cpu(), sample_rate=sr)

In [28]:
display(Audio(mixture[:, start:end].reshape(1,-1), rate = sr))

In [13]:
print(is_silent(original_tracks["4"]))
torch.linalg.norm(original_tracks["4"]) / (end - start)

tensor(True)


tensor(7.4217e-05)

In [32]:
15e2

1500.0